In [150]:
import torch 
import torch.nn as nn
import torch.nn.functional as F 
from torch.autograd import Variable
import numpy as np
import cv2 
import matplotlib.pyplot as plt

In [151]:
def parse_cfg(cfgfile):
    
    file=open(cfgfile, 'r')
    lines=file.read().split('\n')
    lines=[x for x in lines if len(x)>0] #remove empty lines
    lines=[x for x in lines if x[0] !='#'] #remove commented lines
    lines = [x.rstrip().lstrip() for x in lines]
    
    block={}
    blocks=[]
    
    for line in lines:
        if line[0]=='[':
            if len(block) != 0:
                blocks.append(block)
                block = {}
            block["type"] = line[1:-1].rstrip()
            #print(block)
            
        else:
            key, value=line.split('=')
            block[key.rstrip()] = value.lstrip()
    blocks.append(block)
        
    return(blocks)

In [152]:
#parse_cfg('C:/Users/tunes/Downloads/darknet-master/darknet-master/build/darknet/x64/yolov3_face.cfg')

In [153]:
def get_test_input():
    img = cv2.imread("C:/Users/tunes/Downloads/test1.jpg")
    img = cv2.resize(img, (416,416)) 
    img_ =  img[:,:,::-1].transpose((2,0,1))
    img_ = img_[np.newaxis,:,:,:]/255.0
    img_ = torch.from_numpy(img_).float()
    img_ = Variable(img_)
    return img_

In [154]:
class Maxpoolstride1(nn.Module):
    
    def __inti__(self, kernel_size):
        super(Maxpoolstride1, self).__init__()
        self.kernel_size=kernel_size
        self.pad=kernel_size - 1
        
    def forward(self, x):
        padded_x=F.pad(x, (0,self.pad,0,self.pad), mode="replicate")
        pooled_x=nn.MaxPool2d(self.kernel_size, self.pad)(padded_x)
        
        return pooled_x

In [155]:
class Emptylayer(nn.Module):
    
    def __init__(self):
        super(Emptylayer, self).__init__()
       
    def forward(self, x):
        return x

In [156]:
class Detectionlayer(nn.Module):
    
    def __init__(self, anchors):
        super().__init__()
        self.anchors=anchors
        
    def forward(self, x, inp_dim, classes, confidence):
        x=x.data
        global CUDA
        prediction=x
        prediction=predict_transform(prediction, inp_dim, self.anchors, classes, confidence, CUDA)
        return prediction
        

In [157]:
class Upsample(nn.Module):
    
    def __init__(self, stride=2):
        super().__init__()
        self.stride=stride
        
    def forward(self, x):
        stride= self.stride
        assert(x.data.dim() == 4)
        B=x.data.size[0]
        C=x.data.size[1]
        H=x.data.size[2]
        W=x.data.size[3]
        ws=stride
        hs=stride
        x=x.view(B, C, H, 1, W, 1).expand(B, C, H, stride, W, stride).contiguous().view(B, C, H*stride, W*stride)
        
        return x

In [158]:
def Create_modules(blocks):
    
    net_info=blocks[0]
    
    module_list = nn.ModuleList()
    
    index = 0
    
    prev_filters = 3
    
    output_filters = []
    
    for x in blocks:
        module=nn.Sequential()
        
        if x['type'] == 'net':
            continue
            
        if x['type'] == 'convolutional':
            
            activation = x['activation']
            filters = int(x['filters'])
            padding = int(x['pad'])
            kernel_size = int(x['size'])
            stride = int(x['stride'])
            
            try:
                batch_normalize = x['batch_normalize']
                bias = False
                
            except:
                batch_normalize=0
                bias=True
            
            if padding:
                pad = (kernel_size - 1)//2
            else:
                pad = 0
                
            conv=nn.Conv2d(prev_filters, filters, kernel_size, stride, pad, bias= bias)
            module.add_module("conv_{0}".format(index), conv)
            
            
            if batch_normalize:
                bn=nn.BatchNorm2d(filters)
                module.add_module("batch_norm_{0}".format(index), bn)
                
                
            if activation=='leaky':
                activ=nn.LeakyReLU(0.1, inplace=True)
                module.add_module("leaky_{0}".format(index), activ)
                
                
        elif x['type']=='Upsample':
            stride=int(x['stride'])
            upsample=Upsample(stride)
            upsample=nn.Upsample(scale_factor=2, mode='nearest')
            module.add_module("upsample_{0}".format(index), upsample)
            
            
        elif x['type']=='route':
            layers=x['layers'].split(',')
            
            #start of a route
            
            start=int(layers[0])
            
            #end, if exists
            try:
                end=int(layers[1])
                
            except:
                end=0
                
            if start > 0:
                start=start-index
                
            if end > 0:
                end=end-index
                
            route=Emptylayer()
            module.add_module("route_{0}".format(index), route)
            
            
            if end < 0:
                filters=output_filters[index + start] + output_filters[index+end]
                
            else:
                filters=output_filters[index + start]
                
            
        elif x['type']=='shortcut':
            from_=x['from']
            shortcut=Emptylayer()
            module.add_module("shortcut_{}".format(index), shortcut)
            
            
        elif x['type']=='maxpool':
            
            stride = int(x["stride"])
            size = int(x["size"])
            
            if stride != 1:
                
                maxpool = nn.MaxPool2d(size, stride)
                
            else:
                
                maxpool = Maxpoolstride1(size)
            
            module.add_module("maxpool_{}".format(index), maxpool)
                
                
            
            
        elif x['type']=='yolo':
            mask=x['mask'].split(',')
            mask=[int(x) for x in mask]
            
            anchors=x['anchors'].split(',')
            anchors=[int(a) for a in anchors]
            anchors=[(anchors[i], anchors[i+1]) for i in range(0, len(anchors), 2)]
            anchors=[anchors[i] for i in mask]
            
            
            detection=Detectionlayer(anchors)
            module.add_module('detection_{}'.format(index), detection)
            
            
            
            
        
            
            
        module_list.append(module)
        prev_filters = filters
        output_filters.append(filters)
        index += 1
        
    
    return (net_info, module_list)
            
               

In [159]:
class Darknet(nn.Module):
    
    def __init__(self, cfgfile):
        super(Darknet, self).__init__()
        self.blocks = parse_cfg(cfgfile)
        self.net_info, self.module_list = Create_modules(self.blocks)
        self.header = torch.IntTensor([0,0,0,0])
        self.seen = 0

        
        
    def get_blocks(self):
        return self.blocks

    
    def get_module_list(self):
        return self.module_list
    
    
    
    def forward(self, x, CUDA = False):
        
        
        detections=[]
        
        modules = self.blocks[1:]
        outputs={}
        
        
        write=0
    
        
        for i in range(len(modules)):
            #print(len(modules))
            modules_type = (modules[i]['type'])
        
            if modules_type == 'convolution' or modules_type == 'upsample' or modules_type == 'maxpool' :
                x=self.modules_list[i](x)
                print(x)
                outputs[i]=x
                print(outputs[i])
            elif modules_type == 'route':
                layers = modules[i]['layers']
                layers = [int(a) for a in layers]
                
                if layers[0] > 0:
                    layers[0] = layers[0] - i
                    
                if len(layers) == 1:
                    x =  outputs[i + (layers[0])]
                    
                else:
                    if layers[1] > 0:
                        layers[1] = layers[1] - i

                    map1 = outputs [i + layers[0]]
                    map2 = outputs [i + layers[1]]
                    
                    x = torch.cat((map1, map2), 1)
                
                outputs[i] = x
                
            elif modules_type == 'shortcut':
                from_ = int(modules[i]['from'])
                print(from_)
                print(i)
                print(outputs[i-1])
                x = outputs[i-1] + outputs[i + from_]
                outputs[i] = x
                
            
            elif modules_type == 'yolo':
                
                anchors=self.module_list[i][0].anchors
                
                inp_dim=int(self.net_info['height'])
                
                num_classes=int(module[i]['classes'])
                
                x=x.data
                x=predict_transform(x, inp_dim, anchors, num_classes, CUDA)
                
                if type(x) == int:
                    continue
                    
                if not write:
                    detections = x
                    write = 1 
                    
                else:
                    detections = torch.cat((detection, x), 1)
                    
                outputs[i] = outputs[i-1]
                
                
                
        try:
            return detections
        
        except:
            return 0
        
                 
    def load_weights(self, weightfile):
        
        fp = open(weightfile, 'rb')
        
        
        header = np.fromfile(fp, dtype = np.int32, count = 5)
        self.header = torch.from_numpy(header)
        self.seen = self.header[3]
            
        
        weights = np.fromfile(fp, dtype = np.float32)
        
        ptr = 0
        
        
        
        for i in range(len(self.module_list)):
            
            module_type = self.blocks[i+1]['type']
            
            if module_type == 'convolutional':
                model = self.module_list[i]
                
                try:
                    batch_normalize = int(self.blocks[i+1]['batch_normalize'])
                    
                except:
                    batch_normalize = 0
                    
                
                conv = model[0]
                
                if batch_normalize:
                    
                    bn = model[1]
                    
                    num_bn_biases = bn.bias.numel()
                    
                    bn_biases = torch.from_numpy(weights[ptr:ptr + num_bn_biases])
                    ptr += num_bn_biases
                    
                    bn_weights = torch.from_numpy(weights[ptr:ptr + num_bn_biases])
                    ptr += num_bn_biases
                    
                    bn_running_mean = torch.from_numpy(weights[ptr:ptr + num_bn_biases])
                    ptr += num_bn_biases
                    
                    bn_running_var = torch.from_numpy(weights[ptr:ptr + num_bn_biases])
                    ptr += num_bn_biases
                    
                    bn_biases = bn_biases.view_as(bn.bias.data)
                    bn_weights = bn_weights.view_as(bn.weight.data)
                    bn_running_mean = bn_running_mean.view_as(bn.running_mean)
                    bn_running_var = bn_running_var.view_as(bn.running_var)
                    
                    
                    bn.bias.data.copy_(bn_biases)
                    bn.weight.data.copy_(bn_weights)
                    bn.running_mean.copy_(bn_running_mean)
                    bn.running_var.copy_(bn_running_var)
                    
                    
                else:
                    num_biases = conv.bias.numel()
                    
                    conv_biases = torch.from_numpy(weights[ptr:ptr  + num_biases])
                    ptr += num_biases
                    
                    conv_biases = conv_biases.view_as(conv.bias.data)
                    
                    conv.bias.data.copy_(conv_biases)
                    
                
                num_weights = conv.weight.numel()
                
                conv_weights = torch.from_numpy(weights[ptr:ptr + num_weights])
                ptr += num_weights
                #print(conv_weights.shape)
                conv_weights = conv_weights.view_as(conv.weight.data)
                
                conv.weight.data.copy_(conv_weights)
                
                
                    
    def save_weights(self, savedfile, cutoff = 0):
        
        if cutoff <= 0:
            cutoff = len(self.blocks) - 1
            
        fp = open(savedfile, 'wb')
        
        self.header[3] = self.seen
        header = self.header
        
        header = header.numpy()
        header.tofile(fp)
        
        
        for i in range(len(self.module_list)):
            module_type = self.block[i+1]['type']
            
            
            if module_type == 'convolutional':
                model = self.module_list[i]
                
                try:
                    batch_normalize = int(self.block[i+1]['batch_normalize'])
                    
                except:
                    batch_normalize = 0
                
                conv = model[0]
                
                if batch_normalize:
                    bn = model[1]
                    
                    
                    cpu(bn.bias.data).numpy().tofile(fp)
                    cpu(bn.weights.data).numpy().tofile(fp)
                    cpu(bn.running_mean).numpy().tofile(fp)
                    cpu(bn.running_var).numpy().tofile(fp)
                    
                else:
                    cpu(conv.bias.data).numpy().tofile(fp)
                    
                
                cpu(conv.weights.data).numpy().tofile(fp)
                
                
            
                    

In [160]:
a = Darknet('C:/Users/tunes/Downloads/darknet-master/darknet-master/build/darknet/x64/yolov3_face.cfg')

In [161]:
a.load_weights('C:/Users/tunes/Downloads/darknet-master/darknet-master/build/darknet/x64/yolov3.weights')

In [162]:
inp = get_test_input()

In [163]:
s,d = a(inp)

-3
4


KeyError: 3

In [ ]:
#a.eval()

In [165]:
v = Darknet('C:/Users/tunes/Downloads/darknet-master/darknet-master/build/darknet/x64/yolov3_face.cfg')

In [166]:
v.load_weights('C:/Users/tunes/Downloads/darknet-master/darknet-master/build/darknet/x64/yolov3.weights')

In [167]:
inp = get_test_input()

In [168]:
s,d = v(inp)

NameError: name 'predict_transform' is not defined